# Processing Pasadena (Texas) jail logs: 8/18 - 5/21

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import datetime as dt
import matplotlib.pyplot as plt
import altair as alt

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### Read the raw data

The department provided the data in a [messy PDF format](data/raw/Restraint_Chair_Data.pdf). That limits this initial analysis in two ways: 1) some of the pages are scanned as images, rendering them flat (and impervious to PDF-to-text parsing software); and 2) the redactions in the PDF's narrative field make it impossible to extract in a machine readable format. We can talk about strategies for getting around this. It should also be noted that there are roughly 100 missing records because of the flat pages issue. Nevertheless, I was able to extract roughly 500 records from the PDF into a dataframe for analysis. 

In [4]:
# src = pd.read_csv(
#     "data/raw/chair-slices/tabula-chair-columns_0-6.csv",
#     parse_dates=["date"],
#     dtype={"id": "str"},
# )

In [5]:
src = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQFl6SwQ0V_c4hU0R8gpbnUAOQJyfezGdoFpzQabLVYmCpH8wD-oSmU9fCs3qKHXftaNdPp72I3t1sn/pub?output=csv",
    header=None,
    names=[
        "id",
        "date",
        "time",
        "shift",
        "reporting_personnel",
        "number_assisting",
        "location",
        "narrative",
    ],
    parse_dates=["date"],
    dtype={"id": "str", "time": str},
)

In [6]:
src.sort_values("date", ascending=False).head()

,id,date,time,shift,reporting_personnel,number_assisting,location,narrative
606,202100126,2021-05-05,2240,NITE,"KENNEDY, J",3.0,DWI \nROOM,"Prisoner came into the jail intoxicated and was restrained in the chair due to the male needing a blood draw. [05/06/2021 01:08, JEKENNEDY, , PPD] Prisoner arrived to the DWI room and at removal of handcuffs by Ofc Green the inmate threw himself on to \nPAY \nthe ground stating he needs medical. Inmate was observed to have dry blood on his right hand and stated he was in pain and that he was diabetic. Inmate was then checked and further talked to in spanish by PSO Villegas. PSO Villegas advised that he was in pain due to his hand hurting and was given instructions to sit up and to allow for further search of property. Prisoner refused and continued to lay on ground. Officer Green advised that he was going to be a Magistrate Blood Draw and that restraint chair would be required for inmate.Inmate was restrained without further incident. [05/06/2021 01:13, JEKENNEDY, , PPD] \nPAY"
605,202100124,2021-04-30,100,NITE,"VILLEGAS, L",4.0,SALLY \nPORT,"Inmate was restrained in Sally Port at arrival due to being belligerent at arrest location. No further incident occurred. [04/30/2021 03:55, LVILLEGAS, , PPD] \nPA"
604,202100121,2021-04-27,2335,NITE,"VILLEGAS, L",2.0,BOOKING AREA,"INMATE ARRIVED BELLIGERENT AND STARTED PULLING FROM OFFICER HERNANDEZ AND MYSELF. PRISONER WAS INFORMED TO COOPERATE AND SHE CONTINUED TO PULL AND TRY TO STOP FROM ENTERING THE FACILITY. [04/28/2021 01:52, LVILLEGAS, , PPD] \nPA"
603,202100116,2021-04-25,1000,DAYS,"MARROQUIN, J",3.0,BOOKING AREA,"Officer Powell informed Medina,Christopher Booking No 258324 that he would be getting his blood drawn due to his \nDOB\nDWI charge. Medina stated he would not comply. Once Acadian arrived to the jail, Medina was placed in the restraint chair by Officer Jones, Officer Doskocil and myself. There was minimal resistance from Medina during the process. All straps were checked and he was safely secured in the restraint chair. During the blood draw, Medina began to make threats towards the arresting officer claiming he would beat him up once released from the restraint chair. Medina was also verbally abusive towards the medic that drew his blood. Medina remained in the restraint chair for the officer and jail staff safety. PSO Supervisor Davis was notified of the incident. [04/25/2021 13:16, JMARROQUIN, , PPD] \nPA"
602,202100114,2021-04-20,520,NITE,"BETANCOURT, L",3.0,CELL,"At approximately 0520 prisoner Gomez, Lance Giovanni Booking #258265 was behaving erratically and placed in \nDOB\nthe restraint chair after continously hitting the cell door, punching the phone and kicking the toilet on multiple times. Prisoner Gomez refused to stop and became belliegerent after being advised that jail staff would have to place him in a restraint chair for his own safety and to prevent damage to the facility. PSO Supervisor Adame attempted to deescalate the situation but the prisoner again stated that he would not calm down and wanted to be placed in to the restraint chair. Pso Burden, Pso Adame, and myself Pso Betancourt placed the prisoner in to the restraint chair with some resistance but no injuries to jail staff or the inmate. PSO Supervisor Burden was present on scene. [04/20/2021 05:50, LBETANCOURT, , PPD] \nPA"


### Dates

In [7]:
src["date"] = pd.to_datetime(src["date"], format="%m/%d/%Y")
src["date"] = pd.to_datetime(src["date"].dt.strftime("%Y-%m-%d"))
src["year"] = src["date"].dt.year
src["quarter"] = src["date"].dt.quarter
src["day"] = src["date"].dt.day
src["month"] = src["date"].dt.month
src["weekday"] = src["date"].dt.day_name()
src["monthname"] = src["date"].dt.month_name()
src["month_year"] = pd.to_datetime(src["date"]).dt.to_period("M")
src["month_year_full"] = src["date"].apply(lambda x: x.strftime("%B-%Y"))
src["month_year_full"] = pd.to_datetime(src["month_year_full"])

### Times

In [8]:
src["time"] = src["time"].astype(str).str.zfill(4)
src["hour"] = src["time"].str[:2]
src["minute"] = src["time"].str[-2:]
src["time_full"] = pd.to_datetime(
    src["hour"] + ":" + src["minute"], format="%H:%M"
).dt.time
src["hour"] = src["hour"].astype(int)
src["minute"] = src["minute"].astype(int)

In [9]:
src["location"] = src["location"].str.replace("\n", " ", regex=False)

### How many records? 

In [10]:
len(src)

607

### Make a copy of the dataframe

In [11]:
df = src.copy()

---

### How many restraint cases per year? 

In [12]:
years = df.groupby("year")["id"].size().reset_index()
years

,year,id
0,2018,157
1,2019,228
2,2020,166
3,2021,56


### Which weekdays had the most incidents? 

In [13]:
weekdays = df.groupby("weekday").size().reset_index(name="count")
weekdays.sort_values("count", ascending=False)

,weekday,count
2,Saturday,108
3,Sunday,106
6,Wednesday,89
0,Friday,88
5,Tuesday,86
4,Thursday,72
1,Monday,58


### At which hours were these incidents most common? 

In [14]:
hours = df.groupby("hour").size().reset_index(name="count")
hours.sort_values("count", ascending=False)

,hour,count
2,2,45
1,1,44
3,3,43
0,0,39
21,21,39
17,17,38
4,4,33
20,20,32
15,15,32
14,14,32


### Chart the hours

In [15]:
chair_hours = (
    alt.Chart(hours)
    .mark_bar()
    .encode(
        alt.X("hour:O", title="Hour", axis=alt.Axis(grid=False)),
        y=alt.Y(
            "count:Q",
            title="Count",
            axis=alt.Axis(
                tickCount=6,
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
                title="Arrests per month",
            ),
        ),
    )
    .properties(
        width=600, height=330, title="Chair restraint frequency, by hour in day"
    )
)
chair_hours.configure_view(strokeOpacity=0)

alt.Chart(...)

### Which locations were most common? 

In [16]:
locations = df.groupby("location").size().reset_index(name="count")
locations.sort_values("count", ascending=False)

,location,count
2,CELL,222
1,BOOKING AREA,173
6,SALLY PORT,143
0,AFIS ROOM,28
3,DWI ROOM,9
4,JUVENILE HOLDING AREA,3
5,LAUNDRY ROOM,1


### Who logged restraints the most (i.e., who to call)? 

In [17]:
personnel = df.groupby("reporting_personnel").size().reset_index(name="count")
personnel.sort_values("count", ascending=False).head(10)

,reporting_personnel,count
2,"BETANCOURT, L",88
12,"HO, T",56
18,"PINA, F",47
11,"HERRERA, M",47
15,"MARROQUIN, J",40
29,"WALKER, K",31
24,"THOMAS, R",28
0,"ADAME, R",25
6,"CORTES, A",23
17,"OROZCO, C",22


### Which shifts were most common? 

In [18]:
shifts = df.groupby("shift").size().reset_index(name="count")
shifts.sort_values("count", ascending=False).head(10)

,shift,count
2,NITE,297
1,EVES,255
0,DAYS,55


### Incidents per day in 2020

In [19]:
df2020 = df[df["year"] == 2020].reset_index()

In [20]:
days2020 = df2020.groupby("date").size().reset_index(name="count")
days2020.sort_values("count", ascending=False).head(10)

,date,count
46,2020-05-24,4
52,2020-06-10,4
81,2020-09-08,3
91,2020-10-01,3
97,2020-10-14,2
87,2020-09-23,2
95,2020-10-09,2
38,2020-04-29,2
40,2020-05-14,2
43,2020-05-20,2


### How many times a week did this happen? 

In [21]:
(days2020["count"].sum() / 52).round(2)

3.19

---

### Just Carter cases

In [22]:
df["carter"] = df["narrative"].str.lower().str.contains("carter")

In [23]:
carter = df[df["carter"] == True]

In [24]:
carter

,id,date,time,shift,reporting_personnel,number_assisting,location,narrative,year,quarter,day,month,weekday,monthname,month_year,month_year_full,hour,minute,time_full,carter
7,201800034,2018-05-31,2110,EVES,"HO, T",3.0,CELL,"While assigned to the jail position, the inmate BORJON,MARIO RAYMOND Booking#239273 was placed in the restraint chair at approximatley 2110hrs. I was notified by Supervisor Rhodes that the inmate was repeatedley strking the wall in cell 1705. In fear for his safety, we made our way to the inmate`s cell. Pso Teel, and Officer Carter assisted us in placing the inmate in the restraint chair with no further incident or injury. [05/31/2018 21:32, THO, , PPD] \nPA",2018,2,31,5,Thursday,May,2018-05,2018-05-01,21,10,21:10:00,True
15,201800048,2018-06-15,1815,EVES,"HO, T",2.0,SALLY PORT,"At approximately 1815hrs, the inmate MEDINA,HUMBERTO Booking#240586 was placed in the restraint chair. Officer Carter requested for the beligerent inmate to be chaired upon arrival. PSO Betancourt assissted me in chairing the inmate with no further incident nor injury. Supervisor Rhodes was working MC and approved. [06/15/2018 18:46, THO, , PPD] \nPA",2018,2,15,6,Friday,June,2018-06,2018-06-01,18,15,18:15:00,True
18,201800052,2018-06-17,1750,EVES,"BETANCOURT, L",2.0,SALLY PORT,"Prisoner Maestre,Armando Francisco was placed into the restraint chair upon arrival for being belligerent on scene. I PSO Betancourt walked out to the sally por to meet Ofc Carter with the prisoner. We placed the prisoner in the restraint chair and contined to place the prisoner into cell 1708 for further observation. Supervisor Degraff was notified of the incident [06/17/2018 18:01, LBETANCOURT, , PPD] [06/17/2018 18:30, LBETANCOURT, , PPD] \nPA PA",2018,2,17,6,Sunday,June,2018-06,2018-06-01,17,50,17:50:00,True
91,201800138,2018-09-24,0030,NITE,"BETANCOURT, L",2.0,BOOKING AREA,"At aproximately 0030 prisoner Prince,Travis was brougth in to the jail. The prisoner constantly used verbally assaultive language towards officers and jail staff. The prisoner was padded down and placed in the holding cage where he began to hit the cage and said he was going to give the jail staff hell. The prisoner was advised to calm down and to take a seat and wait to be proccesed, but began to jump on the bench. Immediately Ofc Carter and myself PSO Betancourt walked in to the holding cage and told the prisoner to place his hands behind his back. Right away we took control of the prisoner holding his arms behind his back and walked him to the restraint chair. The prisoner was strapped down to the chair by Ofc Carter, PSO Pina, PSO Betancourt, with the prisoner giving some resistance and placed into cell 1708 to be further observed. The prisoner continued to use abusive language and threaten staff as they passed by the cell. PSO Walls was present in the jail when the incident took place, and there was no injuries. [09/24/2018 02:51, LBETANCOURT, , PPD] \nPA",2018,3,24,9,Monday,September,2018-09,2018-09-01,0,30,00:30:00,True
96,201800143,2018-10-02,0245,NITE,"BETANCOURT, L",2.0,CELL,"At approximately 0240 prisoner Rychlik,Ambor began to kick the door complaining that it was cold and that we needed to give her a blanket. The prisoner was advised to stop kicking the door or she would be placed in to a restraint chair which she replied "" I dont give a fuck come and put me in one""! The prisoner contined to kick the door repeatedly. Immediately after PSO Pina, Ofc Carter, and myself PSO Betancourt made our way to cell G2. Upon arrival the prisoner was ordered to stand up, pull her arms out of her shirt and put her hands behind her back. The prisoner refused at first but complied after multiple commands. The prisoner was placed in the restraint chair while still using abusive language. Supervisor Adame was informed of the incident. [10/02/2018 03:22, LBETANCOURT, , \nPA\nPPD]",2018,4,2,10,Tuesday,October,2018-10,2018-10-01,2,45,02:45:00,True
98,201800145,2018-10-05,0510,NITE,"BETANCOURT

---

### Exports

In [25]:
carter.to_csv("data/processed/carter_restraint_cases.csv", index=False)

In [26]:
df.to_csv("data/processed/all_restraint_cases.csv", index=False)

In [27]:
chair_hours.save("visuals/chair_hours_frequency.png", scale_factor=1)